In [1]:
import sys
sys.path.insert(0, "../src")

In [2]:
import re
import yaml
from sariModelInterpreter import modelInterpreter as mi

In [3]:
inputFile = '../models/bso.yml'

In [4]:
model = mi.parseModelFromFile(inputFile)

In [28]:
def getNodeWithId(model, id):
    
    def traverseNode(id, node):
        if node['id'] == id:
            nodeToReturn.append(node)
        elif 'children' in node:
            for child in node['children']:
                traverseNode(id, child)
    
    nodeToReturn = []
    for node in model:
        traverseNode(id, node)
    
    if len(nodeToReturn):
        return nodeToReturn[0]
    else:
        return False

def convertModelToGraph(model):
    def fillGraph(node):
        if 'children' in node:
            graph[node['id']] = [d['id'] for d in node['children']]
            for child in node['children']:
                fillGraph(child)
        else:
            graph[node['id']] = []   
          
    graph = {}  
    for node in model:
        fillGraph(node)
        
    return graph

def findPath(graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return path
    if not start in graph:
        return None
    for node in graph[start]:
        if node not in path:
            newPath = findPath(graph, node, end, path)
            if newPath:
                return newPath
    return None
    
def getPathQuery(model, path):
    
    nodes = {}
    queryPath = ''
    
    for nodeid in path:
        nodes[nodeid] = getNodeWithId(model, nodeid)
        
    prevId = False
    for step in path[1:]:
        query = mi.namespaceVariablesInQuery(nodes[step]['query'], step)
        if prevId:
            query = query.replace("$subject", "?value_" + prevId)
        queryPath += query + "\n"
        prevId = step
    
    return queryPath
    
def compileQueryForNodes(model, rootId, nodeIds):
    root = getNodeWithId(model, rootId)
    graph = convertModelToGraph(model)
    paths = []
    pathQueries = []
    for nodeId in nodeIds:
        paths.append(findPath(graph, rootId, nodeId))
    for path in paths:
        pathQueries.append(getPathQuery(model, path))
        
    query = "SELECT ($subject as ?%s) " % rootId
    for nodeId in nodeIds:
        query += "(?value_%s as ?%s) " % (nodeId, nodeId)
    query += "{ \n" + "\n".join(pathQueries) + "\n}"
    return query
    
print(compileQueryForNodes(model, 'artwork',['work_creator', 'artwork_medium']))


SELECT ($subject as ?artwork) (?value_work_creator as ?work_creator) (?value_artwork_medium as ?artwork_medium) { 
 $subject crm:P128_carries ?value_work . 
 ?value_work crm:P94i_was_created_by ?value_work_creation . 
 ?value_work_creation crm:P14_carried_out_by ?value_work_creator . 

 $subject crm:P2_has_type ?value_artwork_medium . ?value_artwork_medium crm:P2_has_type aat:300014842 ; rdfs:label ?label_artwork_medium . 

}
